---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kevin-crotty.com)
    
Jones Graduate School of Business, Rice University

---


# EXAMPLE DATA

In [1]:
coupon_rate = 0.04              # coupon rate (decimal notation)
coupons_remaining   = 20        # remaining coupons for the bond
days_to_next_coupon = 60        # days to next coupon payment
yld = 0.06                      # annual yield (decimal notation)

# BOND CLASS

In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve


class bond:
    def __init__(self, coupon_rate, coupons_remaining, days_to_next=180):
        self.coupon = 100 * coupon_rate / 2
        self.n = coupons_remaining
        self.days = int((coupons_remaining - 1) * 180 + days_to_next)
        self.days_to_next = days_to_next

    def days_remaining(self, day):
        return self.days - day

    def coupons_remaining(self, day):
        return int(np.ceil((self.days_remaining(day)) / 180))

    def fp(self, day):
        x = 180 - self.days_remaining(day) % 180
        return x / 180 if x < 180 else 0

    def accrued(self, day):
        return self.coupon * self.fp(day)

    def dirty(self, day, yld):
        periods = 1 - self.fp(day) + np.arange(self.coupons_remaining(day))
        pvFactors = 1 / (1 + yld / 2) ** periods
        cashFlows = [self.coupon] * (self.coupons_remaining(day) - 1) + [
            100 + self.coupon
        ]
        return np.sum(pvFactors * cashFlows)

    def clean(self, day, yld):
        return self.dirty(day, yld) - self.accrued(day)

# CALCULATIONS

In [3]:
# initialize bond class
b = bond(coupon_rate, coupons_remaining, days_to_next_coupon)

# calculate data
days = [i for i in range(b.days)]
clean = [b.clean(d, yld) for d in days]
dirty = [b.dirty(d, yld) for d in days]

# FIGURE

In [4]:
import plotly.graph_objects as go
# clean price
string1 = "clean price = %{y:.02f}<extra></extra>"
trace1 = go.Scatter(
    x=days, 
    y=clean, 
    mode="lines", 
    hovertemplate=string1, 
    name="clean price"
)
# dirty price
string2 = "dirty price = %{y:.02f}<extra></extra>"
trace2 = go.Scatter(
    x=days,
    y=dirty,
    mode="lines",
    hovertemplate=string2,
    name="dirty price",
    line=dict(color="blue"),
)
# face value
trace3 = go.Scatter(
    x=days,
    y=[100] * len(days),
    hovertemplate="face value<extra></extra>",
    mode="lines",
    line=dict(color="red", dash="dot"),
    name="face value",
)
fig = go.Figure(trace3)
fig.add_trace(trace2)
fig.add_trace(trace1)

fig.update_layout(
    yaxis_tickformat=",.0f",
    yaxis_tickprefix='$',
    hovermode="x unified",
    xaxis_title="Days",
    template="plotly_white",
    legend=dict(
        yanchor="bottom", 
        y=0.01, 
        xanchor="right", 
        x=0.99)
)
fig.show()